<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9C%A0%EC%98%88%EC%8A%AC_keybert_%EB%8B%B5%EB%B3%80_%EC%83%9D%EC%84%B1_AI_(%EC%A7%84%ED%96%89%EC%A4%91).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 특성 추출 사전학습 모델

- skt/kobert-base-v1 (131k download)
- BM-K/KoSimCSE-roberta (87k download)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# DATA_PATH = "/content/drive/MyDrive/생성 AI 모델링/data/"
DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device




'cuda'

In [4]:
yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi_all.csv")


In [5]:
google_food = pd.read_csv(f"{DATA_PATH}playstore_food_reply_0904.csv")

In [6]:
train= yogiyo[['고객리뷰','사장댓글']]

rename_cols = ['review', 'reply']
train.columns = rename_cols
train.head()

,review,reply
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [7]:
review_list = []
for review in train['review']:
    if isinstance(review, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

<ipython-input-7-6374a3c305ac>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'] = review_list


,review,reply
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [8]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

<ipython-input-8-2764c488e75c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'] = reply_list


,review,reply
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...


- 긍부정지수 추가

In [9]:
train_posneg = pd.read_csv(f"{DATA_PATH}train_posneg.csv")

In [10]:
cols = ['review','Unnamed: 0']
train_posneg = train_posneg.drop(columns=cols)
train_posneg

,sentiment,score
0,긍정,0.667125
1,긍정,0.751605
2,긍정,0.445775
3,긍정,0.741268
4,긍정,0.590259
...,...,...
18681,긍정,0.491652
18682,긍정,0.557971
18683,긍정,0.487945
18684,긍정,0.512616


In [11]:
train = pd.concat([train,train_posneg],axis=1,ignore_index=True)
train.columns = ['review','reply','sentiment','score']

- 임시 3000개 추출

In [13]:
shuffled_train = train.sample(frac=1)[:3000]
shuffled_train

,review,reply,sentiment,score
12928,만족스러운 제품 입니다,감사합니다,긍정,0.572451
1891,약간 날에따라 편차가 있지만 대체적으로 만족하는 중국집,저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,긍정,0.517167
2769,이틀연속으로 시켜먹었네요 그정도로 맛있어요,조형훈족발보쌈도시락을 믿고 주문해주셔서 감사합니다 다음 주문시 더욱 더 좋은 서비스...,긍정,0.791717
3678,괜찮았습니다! 김치전 좋아요,맛있게 드셨나요?! 고객님들에게 자꾸만 생각나고 떠오르는 맛집으로 기억 속에 남고 ...,긍정,0.380454
4924,여기가 마라 맛 제일 진하고 맛있어서 여기서만 시켜 먹어요! 역시나 존맛탱 추천합니...,저팔계 중국 마라탕 덮밥 요리 전문점 입니다. 저희 음식을 맛있게 드시고 소중한 ...,긍정,0.814542
...,...,...,...,...
5971,맛나게 잘먹엇슴돠 감사함돠,저희 목살 맛있게 잘 드셨다니 기쁘고 힘이 납니다. 소중한 리뷰와 저희에게 큰 힘이...,긍정,0.580561
10833,맛있고 배달도 빨랐어요,믿고 찾아주시고 맛있게 드셔주셔서 감사합니다! 고객님을 향한 열정만큼은 절대 변함없...,긍정,0.616511
9844,이렇게 주셔도 남나요? 사장님 ~ 맛있게 잘 먹었습니다 번창하세요 사장님,안녕하세요 부자집 입니다! 주문해주셔서 감사합니다 바쁘신 와중에 별점5점!리뷰 감사...,긍정,0.898623
18339,맛있네요 굳굳,맛있게 드시고 별 다섯개 만점 리뷰 정말 감사드립니다~,긍정,0.580293


In [14]:
shuffled_train.isna().sum()

review       0
reply        0
sentiment    0
score        0
dtype: int64

In [15]:
shuffled_train.shape

(3000, 4)

# Keybert (skt kobert)




In [17]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.4 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23765 sha256=06643602abb5f4ea8da313cd3b250455fae67ffdec159f742d6a3866c393fa19
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=21e616893f15febcdd7153dcff92215eea70b135c2ec7baf03e2301c91c4d6a4
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers


In [21]:
from keybert import KeyBERT
# from transformers import XLNetTokenizer, BertTokenizer, BertModel
from tqdm import tqdm  # tqdm 추가
from transformers import AutoModel, AutoTokenizer


# keybert에 사용할 Kobert 모델 로드
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
model = AutoModel.from_pretrained("skt/kobert-base-v1")

# KeyBERT 모델 초기화
keybert_model = KeyBERT(model=model)

# 리뷰 데이터 (train['review']로 가정)
reviews = shuffled_train['review'] # 셔플로 섞은 3000개에 대해서 키워드 추출함
keywords_list = []

# tqdm을 사용하여 진행 상황 표시
for text in tqdm(reviews):
    keywords = keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
    keywords = [keyword[0] for keyword in keywords]
    keywords_list.append(keywords)

# # 결과 출력
# for idx, keywords in enumerate(keywords_list):
#     print(f"Review {idx + 1} Keywords:", keywords)

result_df = pd.DataFrame({'Keywords': keywords_list})
result_df


100%|██████████| 3000/3000 [02:07<00:00, 23.55it/s]


,Keywords
0,"[입니다, 만족스러운, 제품]"
1,"[중국집, 날에따라, 만족하는, 대체적으로, 있지만, 약간, 편차가]"
2,"[그정도로, 맛있어요, 시켜먹었네요, 이틀연속으로]"
3,"[좋아요, 괜찮았습니다, 김치전]"
4,"[메뉴도, 맛있어서, 추천합니다, 여기서만, 여기가, 다른, 먹어요, 존맛탱, 마라..."
...,...
2995,"[맛나게, 잘먹엇슴돠, 감사함돠]"
2996,"[맛있고, 빨랐어요, 배달도]"
2997,"[사장님, 번창하세요, 맛있게, 먹었습니다, 주셔도, 남나요, 이렇게]"
2998,"[굳굳, 맛있네요]"


In [22]:
result_df.to_csv(f"{DATA_PATH}train_3000_keyword_kobert.csv",index=False,encoding='utf-8')

In [23]:
x = pd.read_csv(f"{DATA_PATH}train_3000_keyword_kobert.csv")

In [24]:
tmp = result_df.reset_index()[['Keywords']]
tmp

,Keywords
0,"[입니다, 만족스러운, 제품]"
1,"[중국집, 날에따라, 만족하는, 대체적으로, 있지만, 약간, 편차가]"
2,"[그정도로, 맛있어요, 시켜먹었네요, 이틀연속으로]"
3,"[좋아요, 괜찮았습니다, 김치전]"
4,"[메뉴도, 맛있어서, 추천합니다, 여기서만, 여기가, 다른, 먹어요, 존맛탱, 마라..."
...,...
2995,"[맛나게, 잘먹엇슴돠, 감사함돠]"
2996,"[맛있고, 빨랐어요, 배달도]"
2997,"[사장님, 번창하세요, 맛있게, 먹었습니다, 주셔도, 남나요, 이렇게]"
2998,"[굳굳, 맛있네요]"


In [25]:
tmp.isna().sum()

Keywords    0
dtype: int64

In [26]:
tmp['Keywords'] = tmp['Keywords'].apply(lambda x: ', '.join(x))

In [27]:
# df = pd.read_csv(f"{DATA_PATH}train_3000_keyword_kobert.csv")

In [28]:
tmp = pd.DataFrame(tmp)
tmp

,Keywords
0,"입니다, 만족스러운, 제품"
1,"중국집, 날에따라, 만족하는, 대체적으로, 있지만, 약간, 편차가"
2,"그정도로, 맛있어요, 시켜먹었네요, 이틀연속으로"
3,"좋아요, 괜찮았습니다, 김치전"
4,"메뉴도, 맛있어서, 추천합니다, 여기서만, 여기가, 다른, 먹어요, 존맛탱, 마라,..."
...,...
2995,"맛나게, 잘먹엇슴돠, 감사함돠"
2996,"맛있고, 빨랐어요, 배달도"
2997,"사장님, 번창하세요, 맛있게, 먹었습니다, 주셔도, 남나요, 이렇게"
2998,"굳굳, 맛있네요"


In [29]:
train = shuffled_train.reset_index()[['review','reply','sentiment','score']]
train

,review,reply,sentiment,score
0,만족스러운 제품 입니다,감사합니다,긍정,0.572451
1,약간 날에따라 편차가 있지만 대체적으로 만족하는 중국집,저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,긍정,0.517167
2,이틀연속으로 시켜먹었네요 그정도로 맛있어요,조형훈족발보쌈도시락을 믿고 주문해주셔서 감사합니다 다음 주문시 더욱 더 좋은 서비스...,긍정,0.791717
3,괜찮았습니다! 김치전 좋아요,맛있게 드셨나요?! 고객님들에게 자꾸만 생각나고 떠오르는 맛집으로 기억 속에 남고 ...,긍정,0.380454
4,여기가 마라 맛 제일 진하고 맛있어서 여기서만 시켜 먹어요! 역시나 존맛탱 추천합니...,저팔계 중국 마라탕 덮밥 요리 전문점 입니다. 저희 음식을 맛있게 드시고 소중한 ...,긍정,0.814542
...,...,...,...,...
2995,맛나게 잘먹엇슴돠 감사함돠,저희 목살 맛있게 잘 드셨다니 기쁘고 힘이 납니다. 소중한 리뷰와 저희에게 큰 힘이...,긍정,0.580561
2996,맛있고 배달도 빨랐어요,믿고 찾아주시고 맛있게 드셔주셔서 감사합니다! 고객님을 향한 열정만큼은 절대 변함없...,긍정,0.616511
2997,이렇게 주셔도 남나요? 사장님 ~ 맛있게 잘 먹었습니다 번창하세요 사장님,안녕하세요 부자집 입니다! 주문해주셔서 감사합니다 바쁘신 와중에 별점5점!리뷰 감사...,긍정,0.898623
2998,맛있네요 굳굳,맛있게 드시고 별 다섯개 만점 리뷰 정말 감사드립니다~,긍정,0.580293


In [30]:
tmp = pd.concat([train, tmp],axis=1,ignore_index=True)

In [31]:
tmp

,0,1,2,3,4
0,만족스러운 제품 입니다,감사합니다,긍정,0.572451,"입니다, 만족스러운, 제품"
1,약간 날에따라 편차가 있지만 대체적으로 만족하는 중국집,저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,긍정,0.517167,"중국집, 날에따라, 만족하는, 대체적으로, 있지만, 약간, 편차가"
2,이틀연속으로 시켜먹었네요 그정도로 맛있어요,조형훈족발보쌈도시락을 믿고 주문해주셔서 감사합니다 다음 주문시 더욱 더 좋은 서비스...,긍정,0.791717,"그정도로, 맛있어요, 시켜먹었네요, 이틀연속으로"
3,괜찮았습니다! 김치전 좋아요,맛있게 드셨나요?! 고객님들에게 자꾸만 생각나고 떠오르는 맛집으로 기억 속에 남고 ...,긍정,0.380454,"좋아요, 괜찮았습니다, 김치전"
4,여기가 마라 맛 제일 진하고 맛있어서 여기서만 시켜 먹어요! 역시나 존맛탱 추천합니...,저팔계 중국 마라탕 덮밥 요리 전문점 입니다. 저희 음식을 맛있게 드시고 소중한 ...,긍정,0.814542,"메뉴도, 맛있어서, 추천합니다, 여기서만, 여기가, 다른, 먹어요, 존맛탱, 마라,..."
...,...,...,...,...,...
2995,맛나게 잘먹엇슴돠 감사함돠,저희 목살 맛있게 잘 드셨다니 기쁘고 힘이 납니다. 소중한 리뷰와 저희에게 큰 힘이...,긍정,0.580561,"맛나게, 잘먹엇슴돠, 감사함돠"
2996,맛있고 배달도 빨랐어요,믿고 찾아주시고 맛있게 드셔주셔서 감사합니다! 고객님을 향한 열정만큼은 절대 변함없...,긍정,0.616511,"맛있고, 빨랐어요, 배달도"
2997,이렇게 주셔도 남나요? 사장님 ~ 맛있게 잘 먹었습니다 번창하세요 사장님,안녕하세요 부자집 입니다! 주문해주셔서 감사합니다 바쁘신 와중에 별점5점!리뷰 감사...,긍정,0.898623,"사장님, 번창하세요, 맛있게, 먹었습니다, 주셔도, 남나요, 이렇게"
2998,맛있네요 굳굳,맛있게 드시고 별 다섯개 만점 리뷰 정말 감사드립니다~,긍정,0.580293,"굳굳, 맛있네요"


- Keyword shuffled train 3000

In [32]:
train.isna().sum()

review       0
reply        0
sentiment    0
score        0
dtype: int64

In [33]:
train_keyword = tmp[[4,1,2,3]]
train_keyword.columns = ['review','reply','sentiment','score']
train_keyword.isna().sum()


review       0
reply        0
sentiment    0
score        0
dtype: int64

In [34]:
train_keyword.to_csv(f'{DATA_PATH}keyword_shuffled_train_3000_0904.csv',index=False,encoding='utf-8')

- Sentence shuffled train 3000

In [35]:
train_original = tmp[[0,1,2,3]]
train_original
train_keyword.isna().sum()

review       0
reply        0
sentiment    0
score        0
dtype: int64

In [36]:
train_original.to_csv(f'{DATA_PATH}original_shuffled_train_3000_0904.csv',index=False,encoding='utf-8')

# ------학습, 검증, 추론

# Keyword shuffled train 3000

In [37]:
x = pd.read_csv(f'{DATA_PATH}keyword_shuffled_train_3000_0904.csv',encoding='utf-8')

In [38]:
train_keyword.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   review     3000 non-null   object 
 1   reply      3000 non-null   object 
 2   sentiment  3000 non-null   object 
 3   score      3000 non-null   float64
dtypes: float64(1), object(3)
memory usage: 93.9+ KB


In [39]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   review     2977 non-null   object 
 1   reply      3000 non-null   object 
 2   sentiment  3000 non-null   object 
 3   score      3000 non-null   float64
dtypes: float64(1), object(3)
memory usage: 93.9+ KB


In [40]:
x.isna().sum() # 저장시에는 review에 결측치 없었는데, 불러오니까 결측치 생김. 인코딩 문제는 아님.형식문제?

review       23
reply         0
sentiment     0
score         0
dtype: int64

## 전처리

In [41]:
train = train_keyword.copy()
train.isnull().sum()

review       0
reply        0
sentiment    0
score        0
dtype: int64

#### 고객 id 처리

In [42]:
train = pd.DataFrame(train)
train['reply'] = train['reply'].str.replace(r'\w+\s*님', '고객님', regex=True)


#### 가게명 처리

In [43]:
ygy_store_lst = ["1988응답하라추억의옛날도시락-방이점", "워커스커피 로스터스", "떡깨비-가락점", "당치땡-헬리오시티점", "우시장국밥-석촌점",  "고돼지-송파점",  "마라의신마라탕&마라샹궈",  "춘리마라탕-송파점",  "손수경의육회담은연어-가락점",  "무국적식탁-송파점",  "청년치킨-삼전점",  "송파만다린",  "배달의쌀국수-송파점",  "닭강점-헬리오시티점",  "토핑폭탄김치찜&김치찌개-송파점",  "조형훈족발보쌈",  "여왕의브런치",  "모어댄버터",  "플러스82-송파점",  "더티베지",  "잭아저씨족발보쌈-본점",  "육회&연어-미아본점",  "갈비민족-본점",  "구구족-성신여대역점",  "피자보이시나-대학로점",  "왕빈자삼파전-미아본점",  "행복한찜닭-성북점",  "스시사소우",  "저팔계&중국&마라탕&덮밥&요리&전문점",  "명작파스타-성북점",  "올데이파스타-동대문구점",  "대한냉면-성북점",  "인생제육",  "고기듬뿍미트박스-성북점",  "피자스웨그-성북점",  "청춘식당묵은지김치찜",  "삽교원조두리곱창",  "뜸들이다-북서울꿈의숲점",  "더팔당매운오징어&닭발 김치찜",  "우리집반찬도시락-혜화막내딸지점",  "수피자",  "마라쿡",  "김포불닭발",  "헬로팬닭갈비&마약볶음밥",  "대찌 대파생고기김치찌개&정육왕국물두루치기",  "구래상회",  "승도리네곱도리탕",  "자성당",  "수락",  "나인곱창",  "어시장",  "돈까스회관",  "호랑이초밥",  "땡초곱창막창",  "피자이탈리",  "텐텐마라탕",  "과일에반하다",  "삼다수",  "병천토속순대",  "최고남제육명가",  "짱닭치킨",  "걸작떡볶이",  "직화삼겹직구삼",  "짜글이가돼었소",  "부대찌개자신있는집",  "토핑폭탄김치찜&김치찌개-부천점",  "1989마라탕-부천점",  "미친피자-부평점",  "천년닭강정-부천부평점",  "수상한삼겹살-부천점",  "수제죽전문점-본가진죽-본점",  "오늘은 분식",  "이끌림마라탕",  "탐나는피자-부평점",  "고기혁명-부천점",  "라화방마라탕-신중동점",  "전금례닭볶음탕-부천점",  "큰아들백순대",  "공주닭발-부천시청점",  "신자매김치찜&김치찌개",  "짱닭치킨-도촌점",  "이태리면가게-야탑점",  "폴트버거-판교점",  "랜돌프뉴욕페페로니-야탑점",  "홍싸롱-수제돈까스&파스타",  "쇼부덮밥",  "카산도-정자본점",  "부성초밥-미금본점",  "닭장수섭삼계탕&전기구이통닭",  "깨돌이김밥-미금점",  "울진죽변항",  "GTS버거-분당정자점",  "코브라독스-분당미금역점",  "마라하오-죽전점",  "땅끝수산",  "초밥대통령-용인수지점",  "올데이케밥&샐러드",  "야키토리 카마쿠라",  "비오키친-서현본점",  "부산오빠1인국밥-분당점",  "알촌",  "밈피자",  "고씨네카레",  "홍스족발",  "뜸들이다",  "감성커피",  "짚신스시",  "떡형",  "39도시락",  "피제이피자",  "황궁쟁반짜장",  "OK반점",  "모모타코야키&카페",  "꼬알라파이",  "진지덮밥",  "피자프라텔로",  "한식세끼1인김치찜&김치찌개",  "윤희횟집",  "시민보쌈족발&감자탕",  "카페봄봄",  "킹프레소빅와플",  "최가네한쌈",  "창타이누들",  "보돌미역",  "홍대개미"  ]


In [44]:
store_name = []

for store in ygy_store_lst:
    pat = re.split(r'[- &]', store)
    store_name.append(pat[0])

len(store_name)

125

In [45]:
store_name[:20]

['1988응답하라추억의옛날도시락',
 '워커스커피',
 '떡깨비',
 '당치땡',
 '우시장국밥',
 '고돼지',
 '마라의신마라탕',
 '춘리마라탕',
 '손수경의육회담은연어',
 '무국적식탁',
 '청년치킨',
 '송파만다린',
 '배달의쌀국수',
 '닭강점',
 '토핑폭탄김치찜',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82',
 '더티베지']

In [46]:
# train['store'] = ""  # full store name을 칼럼에 담기
# for name in store_name:
#     train[name] = train['reply'].apply(lambda x: name if name in x else '') # 우선 각 가게명 컬럼 생성(그렇게해야 덮어쓰기가 안됨)
#     train['store'] = train.apply(lambda row: row['store'] + ',' + row[name] if row[name] else row['store'], axis=1) # 하나로 합치기(맨 앞에 콤마가 생김)

# train['store'] = train['store'].apply(lambda x: x[1:] if x.startswith(',') else x) # 맨 앞에 콤마 제거
# train = train.drop(columns=store_name) # 각 가게명 컬럼 삭제

# train.head()

- 1차 : store 이름이 풀로 들어간 경우 <store>로 대체

In [47]:
def replace_store_names1(text):
    for name1 in store_name:
        if name1 in text:
            # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<store>", text)
    return text

In [48]:
replace_store_names1("1988응답하라추억의옛날도시락에서 먹은 음식은 정말 맛있었습니다.")

'<store>에서 먹은 음식은 정말 맛있었습니다.'

In [49]:
replace_store_names1("킹프레소에서 먹은 음식은 정말 맛있었습니다.") # 안바뀜

'킹프레소에서 먹은 음식은 정말 맛있었습니다.'

- 2차 : store 이름이 부분적으로 들어간 경우 store로 대체

In [50]:
def replace_store_names2(text):
    for name1 in tqdm(store_name, desc="Processing"):
        for i in range(len(name1), 1, -1):
            part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
            if part in text:
                # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
                pattern = re.compile(re.escape(part), re.IGNORECASE)
                text = pattern.sub("<store>", text)
    return text


In [51]:
replace_store_names2("텐텐에서 먹은 음식은 정말 맛있었습니다.") # 텐텐마라탕

Processing: 100%|██████████| 125/125 [00:00<00:00, 162017.31it/s]


'<store>에서 먹은 음식은 정말 맛있었습니다.'

In [52]:
replace_store_names2("손수경에서 먹은 음식은 정말 맛있었습니다.") # 손수경의육회담은연어

Processing: 100%|██████████| 125/125 [00:00<00:00, 225306.40it/s]


'<store>에서 먹은 음식은 정말 맛있었습니다.'

- 3차 : 일반명사 등은 제외하고 대체해야 함

In [53]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813255 sha256=0d2a2a71bc2e624435171fd7a5c69ad91c25cc410091fda40a2569cf7a92195e
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


In [54]:
# # short_name 에서 가게명 이외 일상용어로 사용되는 키워드는 제거
# drop_name = ['고기','OK','감성','걸작','공주','과일','김포','대한','땡초','명작','미친','배달','병천','부대','부산','비오','삽교',
#              '손수','송파','쇼부','수상','수제','시민','오늘', '오늘은', '우리', '울진', '이끌', '이끌림', '인생', '직화', '진지',
#              '천년', '청년', '청춘', '최고', '큰아', '큰아들', '탐나', '토핑', '행복', '행복한', '헬로', '홍대',
#              '육회','수락','호랑','갈비','무국','닭장','한식','마라','스시','여왕','돈까','초밥','카페','피자','부대찌','이태리','플러','플러스']
# # drop_name = ['과일','오늘', '오늘은','우리', '이끌', '이끌림', '인생','진지','최고''토핑', '행복', '행복한', '한식', '감성','걸작']

In [55]:
import kiwipiepy
kiwi = kiwipiepy.Kiwi()

In [56]:
def remove_adjectives_nouns(text):
    result = []
    tokens = kiwi.extract_words(text, min_cnt=1, min_score=0.25)
    for token in tokens:
        word = token[0].form  # 단어만 추출하여 사용
        pos = token[1]
        # 형용사나 명사가 아닌 경우에만 결과에 추가
        if pos not in ('VA', 'NNG'):
            result.append(word)
    return ' '.join(result)


In [57]:
tokens = kiwi.analyze(store_name)

remove_words = []

for token in tokens :
  word = token[0][0][0].form
  pos = token[0][0][0].tag
  if pos in ('NNG') and len(word) ==2 :
    print(word)
    remove_words.append(word)

# 2글자 일반명사 단어들은 제외 -> 완벽하지 않아서 세부적인 튜닝 필요. 대찌, 구래, 삼다 등

춘리
청년
배달
토핑
조형
여왕
모어
육회
갈비
피자
행복
명작
인생
고기
청춘
헬로
대찌
구래
자성
수락
나인
땡초
피자
과일
삼다
최고
걸작
직화
토핑
천년
수제
오늘
고기
공주
홍싸
쇼부
부성
초밥
감성
짚신
진지
한식
시민
카페
보도


In [58]:
tokens = kiwi.analyze(store_name)

for token in tokens :
  word = token[0][0][0].form
  pos = token[0][0][0].tag
  if pos in ('NNG') and len(word) ==3 :
    print(word)

# 3글자 일반명사 단어들은 가게명으로 포함해도될듯

떡깨비
우시장
플러스
더티베
돈까스
호랑이
라화방
큰아들
피제이


In [59]:
tokens = kiwi.analyze(store_name)

for token in tokens :
  word = token[0][0][0].form
  pos = token[0][0][0].tag
  if pos in ('NNG') and len(word) ==4 :
    print(word)

# 4글자 일반명사 단어들은 가게명으로 포함해도될듯

부대찌개
폴트버거
창타이누


In [60]:
store_name[:5]

['1988응답하라추억의옛날도시락', '워커스커피', '떡깨비', '당치땡', '우시장국밥']

- 최종 합수

In [61]:
# def replace_store_names(text):
#     for name1 in store_name:
#         # 먼저 가게 이름 전체가 텍스트에 포함되어 있는지 확인
#         if name1 in text:
#             # 가게 이름 전체가 텍스트에 포함되어 있다면 대체
#             pattern = re.compile(re.escape(name1), re.IGNORECASE)
#             text = pattern.sub("<store>", text)
#         else:
#             # 가게 이름의 일부가 텍스트에 포함되어 있는 경우를 확인
#             for i in range(len(name1), 1, -1):
#                 part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
#                 if part in text:
#                     # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
#                     pattern = re.compile(re.escape(part), re.IGNORECASE)
#                     text = pattern.sub("<store>", text)
#                     break  # 가게 이름 일부를 대체했으면 루프 종료
#     return text


In [62]:
def replace_store_names(text):
    for name1 in store_name:
        # 먼저 가게 이름 전체가 텍스트에 포함되어 있는지 확인
        if name1 in text:
            # 가게 이름 전체가 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<store>", text)
        else:
            # 가게 이름의 일부가 텍스트에 포함되어 있는 경우를 확인
            for i in range(len(name1), 1, -1):
                part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
                if part in text:
                    # 가게 이름의 일부가 텍스트에 포함되어 있고, 해당 부분이 removelist에 없는 경우에만 대체
                    if part not in remove_words:
                        pattern = re.compile(re.escape(part), re.IGNORECASE)
                        text = pattern.sub("<store>", text)
                    break  # 가게 이름 일부를 대체했으면 루프 종료
    return text


In [63]:
remove_words[:10]

['춘리', '청년', '배달', '토핑', '조형', '여왕', '모어', '육회', '갈비', '피자']

In [64]:
korean_particles = ['은', '는', '이', '가', '을', '를', '의', '한', '로', '으로', '까지', '이며', '며', '와', '과']

#위 일반명사에, 조사를 합친 경우도 제외함

In [65]:
remove_withparticles = []

for word in remove_words :
  for particle in korean_particles:
    new_word = word + particle
    remove_withparticles.append(new_word)

In [66]:
def replace_store_names(text):
    for name1 in store_name:
        # 먼저 가게 이름 전체가 텍스트에 포함되어 있는지 확인
        if name1 in text:
            # 가게 이름 전체가 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<store>", text)
        else:
            # 가게 이름의 일부가 텍스트에 포함되어 있는 경우를 확인
            for i in range(len(name1), 1, -1):
                part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
                if part in text:
                    # 가게 이름의 일부가 텍스트에 포함되어 있고, 해당 부분이 remove_words에 없으며,
                    # 해당 부분 뒤에 조사가 붙지 않은 경우에만 대체
                    if part not in remove_words + remove_withparticles :
                        pattern = re.compile(re.escape(part), re.IGNORECASE)
                        text = pattern.sub("<store>", text)
                    break  # 가게 이름 일부를 대체했으면 루프 종료
    return text


In [67]:
replace_store_names('행복한 라화방마라탕에서 맛있게 식사했어요') #행복 부분이 대체안됨

'행복한 <store><store>탕에서 맛있게 식사했어요'

In [68]:
# train['reply']의 답변들에 함수 적용하기 / 18000개 7분 정도 걸림

for i, review in tqdm(enumerate(train['reply']), desc="Processing", ncols=100):
    train['reply'][i] = replace_store_names(review)

Processing: 0it [00:00, ?it/s]<ipython-input-68-262f07c68a42>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'][i] = replace_store_names(review)
Processing: 3000it [00:01, 1590.82it/s]


In [69]:
changed_rows = train[train['reply'].str.contains("<store>")] #총 9446개
changed_rows

,review,reply,sentiment,score
2,"그정도로, 맛있어요, 시켜먹었네요, 이틀연속으로",<store>도시락을 믿고 주문해주셔서 감사합니다 다음 주문시 더욱 더 좋은 서비스...,긍정,0.791717
4,"메뉴도, 맛있어서, 추천합니다, 여기서만, 여기가, 다른, 먹어요, 존맛탱, 마라,...",<store> 중국 <store>탕 덮밥 요리 전문점 입니다. 저희 음식을 맛있게...,긍정,0.814542
5,"좋은고같아요, 추천합니다, 맛있어요",맛있게 드셔 주시고 소중한 리뷰 남겨주셔서 감사합니다~ 만족스러운 식사 하신 것 같...,긍정,0.601344
8,"맛볼수있는, 맛있고, 좋네요, 많아요, 엄청, 구성이라, 골고루, 양도",고객님 맛있게 잘 드셨나요? 더블세트를 드셨네요 더블세트는 양도 많고 골고루 드실수...,긍정,0.678257
10,"맛잇게, 잘먹엇습니다",고객님 안녕하세요! 소중한 리뷰 너무 감사드려요 더욱 더 만족스러운 식사를 하실수 ...,긍정,0.546234
...,...,...,...,...
2992,"먹었어요, 맛있게, 치즈양이, 한가득입니다, 스파게티, 올라가는, 위에, 기본",안안녕하세요 <store> 부평점 입니다 D 앞으로도 좋은 재료와 서비스로 고객님께...,긍정,0.523967
2993,"맛있어요, 완벽한, 조합입니다, 너무",고객님 안녕하세요! 소중한 리뷰 너무 감사드려요 더욱 더 만족스러운 식사를 하실수 ...,긍정,0.808070
2994,맛있어요,<store> 족발보쌈을 찾아주셔서 감사합니다 요즘같이 어려운 시기에 주문해주셔서 ...,긍정,0.586300
2997,"사장님, 번창하세요, 맛있게, 먹었습니다, 주셔도, 남나요, 이렇게",안녕하세요 부자집 입니다! 주문해주셔서 감사합니다 바쁘신 와중에 별점5점!리뷰 감사...,긍정,0.898623


In [70]:
# train['review']의 리뷰들에 함수 적용하기 / 7분 정도 걸림

for i, review in tqdm(enumerate(train['review']), desc="Processing", ncols=100):
    train['review'][i] = replace_store_names(review)

Processing: 0it [00:00, ?it/s]<ipython-input-70-1e0bf143e62a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'][i] = replace_store_names(review)
Processing: 3000it [00:01, 2002.53it/s]


In [71]:
changed_rows = train[train['review'].str.contains("<store>")] #총 25개
changed_rows

,review,reply,sentiment,score
4,"메뉴도, 맛있어서, 추천합니다, 여기서만, 여기가, 다른, 먹어요, 존맛탱, <st...",<store> 중국 <store>탕 덮밥 요리 전문점 입니다. 저희 음식을 맛있게...,긍정,0.814542
7,"맛있어요, 알리오파스타는, 좋고, 맛잇어요, 음식도, <store>가, 받으세용, ...",Wow 넘나 긍정적이고 밝은 리뷰말씀 감사해요~ 이렇게 응원해주시는 고객님 기운받아...,긍정,0.910900
9,"맛있게, 먹었습니당, 정말, 굿굿, <store>, 양도",리뷰 감사합니다 전자저울로 정량 드립니다 앞으로도 많이 애용해주세요,긍정,0.567126
11,"먹었습니다, 그래도, 모셨나봐요, 들어주셨는데, 요청사항을, <store>, 항상",죄송합니다 ! 다음에도 애용해주세용,긍정,0.557971
39,"먹고있는, 맛있게, 먹고있어요, 재료도, <store>탕집이에요, 구수하고, 푸짐하...",반갑습니다 맛있게 드셨다니 마음이 놓입니다! 맛 점검은 주기적으로 하며 더 좋은 서...,긍정,0.834987
...,...,...,...,...
2888,"수박쥬스도, 맛있어요, 맛있었어요, 맛집이라고, <store>딸이, 신선해서, 이쁘...",고객님 주문해 주셔서 감사합니다 고객님이 진정 맛잘알 이로군요 주스도 저희가 직접만...,긍정,0.873576
2897,"기<store>거보다, 기대하고, 먹었는데, 맛있어서, 놀람",맛있게 드셔주셔서 너무 감사드립니당~ 연휴 끝나구 피곤하실텐데 피로회복 필요하실 땐...,긍정,0.578536
2951,"<store>의, 여행왔다가, 맛에, 갑니다, 포장주문해서, 타지에서, 감탄하고, ...",저희 <store>을 찾아주셔서 감사드립니다 고객님의 정성스러운 리뷰에 감동입니다 ...,긍정,0.677050
2977,"서울에서, <store>입니다, 족발은, 맛있는, 먹었어요, 가장",<store> 족발보쌈을 찾아주셔서 감사합니다 주문해주셔서 너무 고맙습니다! 푸짐하...,긍정,0.783227


#### 지점명 처리

In [72]:
# 정규 표현식을 사용하여 "~호점" 패턴을 찾아서 삭제하는 함수
def remove_hojum(text):
    return re.sub(r'\w+\s*호점','', text)

# train 데이터프레임의 'reply' 열에 적용하여 "~호점"을 삭제
train['reply'] = train['reply'].apply(remove_hojum)
train['review'] = train['review'].apply(remove_hojum)


In [73]:
restaurant_list = pd.read_csv(f"{DATA_PATH}fulldata_07_24_04_P_일반음식점.csv",encoding='cp949')

<ipython-input-73-afda4487c689>:1: DtypeWarning: Columns (11,27,39,41,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  restaurant_list = pd.read_csv(f"{DATA_PATH}fulldata_07_24_04_P_일반음식점.csv",encoding='cp949')


In [74]:
restaurant_list['소재지전체주소']

0                   서울특별시 동대문구 장안동 382-1
1               서울특별시 마포구 망원동 386-2번지 지층
2                 서울특별시 마포구 서교동 403-14번지
3              서울특별시 마포구 서교동 363-5번지 B1동
4          경기도 안양시 동안구 호계동 930-30번지 지상1층
                       ...              
2086169             광주광역시 서구 용두동 306번지 1
2086170              광주광역시 남구 양림동 374 2층
2086171     광주광역시 남구 월산동 1048-182번지 (1층)
2086172        광주광역시 서구 화정동 332-9번지 (1층)
2086173          광주광역시 서구 화정동 70-61 (1층)
Name: 소재지전체주소, Length: 2086174, dtype: object

In [75]:
address_parts=[] #소재지 데이터에서 서울, 부산 동, 구, 리 등을

for address in restaurant_list['소재지전체주소'].astype(str):
    if address.startswith('서울'or '부산'):
        parts = address.split(' ')
        for _ in range(0, min(4, len(parts))):  # 최대 5개 또는 주소 부분 개수만큼 반복
          part = parts[_]
          address_parts.append(part)

address_parts = set(address_parts)
address_parts = list(address_parts)

address_parts = [name for name in address_parts if all(char.isalpha() for char in name)] #알파벳으로만 (한글) 구성된 경우남김. 숫자, 특수문자 지움
address_parts.remove('')

In [76]:
# 가게명에서 지점명 추출하기

store_etc_name = []

for store in ygy_store_lst:
    pat = re.split(r'[- &]', store)
    for name in pat[1:]:
        if name != '':
            store_etc_name.append(name)

ygy_region_name = []

for region in store_etc_name:
  if region[-1] =='점':
    ygy_region_name.append(region)

In [77]:
address_parts2 = [] # 추출된 지역명을 지점명으로 변경

for address in address_parts :
  if len(address) < 3  :
    address = address + '점'
    address_parts2.append(address)
  else :
    address = address[:-1] + '점'
    address_parts2.append(address)

In [78]:
len(address_parts2)

359

In [79]:
region_name = ygy_region_name + address_parts2

In [80]:
region_name = set(region_name)
region_name = list(region_name)

In [81]:
region_name[:10]

['북아현점', '남창점', '반포점', '금천점', '대림점', '하왕십리점', '송정점', '서울특별점', '홍은점', '은평점']

- 지점명의 경우 1, 2, 3차로 나누지 않고 지점명 풀로 포함된 경우에만 대체 (~점 포함, 기본 2글자)

In [82]:
def replace_region_names(text):
    for name1 in region_name:
        if name1 in text:
            # 지점 이름이 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<region>", text)
        # else :
        #   for i in range(len(name1), 1, -1):
        #       part = name1[:i]  # 지점 이름에서 처음부터 i 글자까지의 부분
        #       if part in text:
        #           # 지점 이름의 일부가 텍스트에 포함되어 있다면 대체
        #           pattern = re.compile(re.escape(part), re.IGNORECASE)
        #           text = pattern.sub("<region>", text)
    return text


In [83]:
replace_region_names('오랜만에 헬리오시티점에서 맛있게 식사했어요')

'오랜만에 <region>에서 맛있게 식사했어요'

In [84]:
for i, review in tqdm(enumerate(train['reply']), desc="Processing", ncols=100):
    train['reply'][i] = replace_region_names(review)

Processing: 0it [00:00, ?it/s]<ipython-input-84-bf597695a038>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'][i] = replace_region_names(review)
Processing: 3000it [00:01, 2547.51it/s]


In [85]:
changed_rows = train[train['reply'].str.contains("<region>")]
changed_rows

,review,reply,sentiment,score
4,"메뉴도, 맛있어서, 추천합니다, 여기서만, 여기가, 다른, 먹어요, 존맛탱, <st...",<store> 중국 <store>탕 덮밥 요리 <region> 입니다. 저희 음식...,긍정,0.814542
22,"먹었습니다, 맛있게, 맵습니다, 양도, 많고, 그래도, 1단계여도, 맵기",저희 <store> <region>을 찾아주셔서 정말 대단히 감사합니다 이렇게 맛있...,긍정,0.485288
30,"맛있어요, 고추, 음식이, 맛있어보여, 최고입니다, 요구르트사진만, 섞어버려, 아보...","고객님, 안녕하세요! 항상 고객님께 최선을 다하는 <store> <region> 입...",긍정,0.822734
41,"맛있게, 먹었어요, 따뜻하게, 진짜, 놀랐어요, 빨라서, 배달",최<store> 따뜻하고 맛있게 보내드리기위해 <store> <region>의 주인...,긍정,0.622591
56,"좋은, 좋았습니다, 이벤트도, 맛이, 하고",안녕하세요 <store> <region> 입니다 D 앞으로도 좋은 재료와 서비스로 ...,긍정,0.562911
...,...,...,...,...
2931,"맛있어요, 너무",안녕하세요 <store> <region> 입니다 D 소중한 리뷰와 별점 너무 감사드...,긍정,0.822610
2939,맛있어요,식사는 맛있게 하셨나요? 혜화 대학로 SNS 피자랭킹맛집 <store> <regi...,긍정,0.571804
2946,"맛나게, 먹었어용, 너무",안녕하세요 <store> <region> 입니다 D 앞으로도 좋은 재료와 서비스로 ...,긍정,0.665527
2974,"가게에요, 시킬, 믿고, 있는",안녕하세요 <store>끼 청주점입니다 맛있게 드셨다니 열심히 구운 보람을 느껴요 ...,긍정,0.637445


In [86]:
for i, review in tqdm(enumerate(train['review']), desc="Processing", ncols=100):
    train['review'][i] = replace_region_names(review)

Processing: 0it [00:00, ?it/s]<ipython-input-86-f126a8c55200>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'][i] = replace_region_names(review)
Processing: 3000it [00:01, 2549.49it/s]


In [87]:
changed_rows = train[train['review'].str.contains("<region>")]
changed_rows

,review,reply,sentiment,score
1156,"맛있어요, 먹었습니다, 연어, 참치도, 맛나게, 별미네요, <store>에서, 육사...",맛있게 드셔주셔서 감사드립니다 늘 믿고 드실 수 있게 항상 신선한 재료로 정성을 듬...,긍정,0.656358
1575,"너무맛있어요, 여기서만, 번창하세요, <region>에서만, <store>비빔밥은,...","안녕하세요! 잊지 않으시고 저희 매장에 발자취 남겨주셔서 감사합니다 언제나 양, 맛...",긍정,0.889644
2725,"<region>이, 식당은, 성북구, 맛있습니다, 강북구, 배달예정시간도, 다른지점...","고객님, 안녕하세요! 항상 고객님께 최선을 다하는 <store> <region> 입...",부정,0.369069


#### 메뉴명 처리

In [88]:
food_list = pd.read_csv(f'{DATA_PATH}전국통합식품영양성분정보(음식)표준데이터.csv', encoding='cp949')

In [89]:
food_list = set(food_list['대표식품명'].tolist())

In [90]:
food_list = list(food_list)

In [91]:
ygy_menu_list = ['양식','일식','중식','중국집','태국음식','한식','분식','도시락','카레','커피','국밥','마라탕','마라샹궈','우동','나가사키','연어','치킨','쌀국수','닭강정','김치찜','김치찌개',
                '족발','보쌈','육회','갈비','피자','찜닭','파스타','냉면','곱창','닭발','막창','반찬','곱도리탕','순대','떡볶이','부대찌개'
                '삼겹살','분식','라면','버거','덮밥','초밥','통닭','짜장','짬뽕','탕수육','카페','와플','누들','김치','온면','디저트','라떼','고기','덮밥',
             '새우','돼지','닭','찌개','밥','타코야키','탕',
                 '양파','숙주','고수'] # 재료명 제거

In [92]:
menu_name = food_list + ygy_menu_list

In [93]:
len(menu_name)

822

In [94]:
new_food_list = []  # 새로운 음식을 담을 리스트

for food in food_list:
    if '(' in food:
        food1, food2 = food.split('(')
        new_food_list.append(food1.strip())  # 공백 제거 후 리스트에 추가
        new_food_list.append(food2.rstrip(')'))
    else:
        new_food_list.append(food)

food_list = new_food_list

In [95]:
menu_name = set(menu_name)
menu_name = list(menu_name)

menu_name = [name for name in menu_name if all(char.isalpha() for char in name)] #알파벳으로만 (한글) 구성된 경우남김. 숫자, 특수문자 지움


- 1차 : 풀로 이름 들어간 것 대체

In [96]:
def replace_menu_names1(text):
    # 메뉴 이름을 긴 것부터 짧은 것 순으로 정렬
    sorted_menu_names = sorted(menu_name, key=len, reverse=True)

    for name1 in sorted_menu_names:
        if name1 in text:
            # 메뉴 이름이 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<menu>", text)
    return text


In [97]:
replace_menu_names1('부대찌개 정말 맛있었어요')

'<menu> 정말 맛있었어요'

- 2차 : 부분 이름 들어간 것 대체

In [98]:
menu_name[:10]

['미꾸라지튀김',
 '브로콜리볶음',
 '잔멸치볶음',
 '호박오가리나물무침',
 '충무김밥',
 '코다리찜',
 '허니브레드',
 '대추차',
 '주먹밥',
 '우동볶음']

In [99]:
name1='단무지무침'

for i in range(len(name1),1,-1):
    part = name1[:i]
    print(part)

단무지무침
단무지무
단무지
단무


In [100]:
def replace_menu_names2(text):

    for name1 in menu_name:
        for i in range(len(name1), 1, -1):
            part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
            if part in text:
                # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
                pattern = re.compile(re.escape(part), re.IGNORECASE)
                text = pattern.sub("<menu>", text)

    return text


In [101]:
replace_menu_names2('부대 정말 맛있었어요')

'<menu> 정말 맛있었어요'

- 3차 : 일반명사 제외 안해도됨. 재료명 모두 일반명사

- 최종 함수

In [102]:
def replace_menu_names(text):
    # 메뉴 이름을 긴 것부터 짧은 것 순으로 정렬
    sorted_menu_names = sorted(menu_name, key=len, reverse=True)

    for name1 in sorted_menu_names:
        if name1 in text:
            # 메뉴 이름이 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<menu>", text)
        else:
            for i in range(len(name1), 1, -1):
                part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
                if part in text:
                    # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
                    pattern = re.compile(re.escape(part), re.IGNORECASE)
                    text = pattern.sub("<menu>", text)
                    break  # 대체를 하고 나서는 루프를 종료합니다.
    return text


In [103]:
# def replace_menu_names(text):
#     # 메뉴 이름을 긴 것부터 짧은 것 순으로 정렬
#     if text is not None:
#         sorted_menu_names = sorted(menu_name, key=len, reverse=True)

#         for name1 in sorted_menu_names:
#             if name1 in text:
#                 # 메뉴 이름이 텍스트에 포함되어 있다면 대체
#                 pattern = re.compile(re.escape(name1), re.IGNORECASE)
#                 text = pattern.sub("<menu>", text)
#             else:
#                 for name1 in menu_name:
#                     for i in range(len(name1), 1, -1):
#                         part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
#                         if part in text:
#                             # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
#                             pattern = re.compile(re.escape(part), re.IGNORECASE)
#                             text = pattern.sub("<menu>", text)
#                         break

In [104]:
for i, review in tqdm(enumerate(train['reply']), desc="Processing", ncols=100):
    train['reply'][i] = replace_menu_names(review)

Processing: 0it [00:00, ?it/s]<ipython-input-104-9d86960b7397>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'][i] = replace_menu_names(review)
Processing: 3000it [00:05, 596.44it/s]


In [105]:
changed_rows = train[train['reply'].str.contains("<menu>")]
changed_rows

,review,reply,sentiment,score
1,"중국집, 날에따라, 만족하는, 대체적으로, 있지만, 약간, 편차가",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,긍정,0.517167
2,"그정도로, 맛있어요, 시켜먹었네요, 이틀연속으로",<store><menu>을 믿고 주문해주셔서 감사합니다 다음 주문시 더욱 더 좋은 ...,긍정,0.791717
4,"메뉴도, 맛있어서, 추천합니다, 여기서만, 여기가, 다른, 먹어요, 존맛탱, <st...",<store> <menu> <store>탕 <menu> 요리 <region> 입니...,긍정,0.814542
6,"고기가, 맛있어요, 너무, 덮밥이, 적어요, 양에, 비해",<menu>양을 쪼끔이라도 더 늘려볼게용 감사합니당,부정,0.440357
8,"맛볼수있는, 맛있고, 좋네요, 많아요, 엄청, 구성이라, 골고루, 양도",고객님 맛있게 잘 드셨나요? 더블세트를 드셨네요 더블세트는 양도 많고 골고루 드실수...,긍정,0.678257
...,...,...,...,...
2992,"먹었어요, 맛있게, 치즈양이, 한가득입니다, 스파게티, 올라가는, 위에, 기본",안안녕하세요 <store> <region> 입니다 D 앞으로도 좋은 재료와 서비스로...,긍정,0.523967
2993,"맛있어요, 완벽한, 조합입니다, 너무",고객님 안녕하세요! 소중한 리뷰 너무 감사드려요 더욱 더 만족스러운 식사를 하실수 ...,긍정,0.808070
2994,맛있어요,<store> <menu><menu>을 찾아주셔서 감사합니다 요즘같이 어려운 시기에...,긍정,0.586300
2997,"사장님, 번창하세요, 맛있게, 먹었습니다, 주셔도, 남나요, 이렇게",안녕하세요 부자집 입니다! 주문해주셔서 감사합니다 바쁘신 와중에 별점5점!리뷰 감사...,긍정,0.898623


In [106]:
for menu in menu_name:
    for i, review in enumerate(train['review']):
        if menu in review:
            train['review'][i] = replace_menu_names(review)

<ipython-input-106-68d2cd102cee>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'][i] = replace_menu_names(review)


In [107]:
changed_rows = train[train['review'].str.contains("<menu>")]
changed_rows

,review,reply,sentiment,score
1,"<menu>, 날에따라, 만족하는, 대체적으로, 있지만, 약간, 편차가",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,긍정,0.517167
3,"좋아요, 괜찮았습니다, <menu>",맛있게 드셨나요?! 고객님들에게 자꾸만 생각나고 떠오르는 맛집으로 기억 속에 남고 ...,긍정,0.380454
6,"<menu>가, 맛있어요, 너무, <menu>이, 적어요, 양에, 비해",<menu>양을 쪼끔이라도 더 늘려볼게용 감사합니당,부정,0.440357
7,"맛있어요, 알리오<menu>는, 좋고, 맛잇어요, 음식도, <store>가, 받으세...",Wow 넘나 긍정적이고 밝은 리뷰말씀 감사해요~ 이렇게 응원해주시는 고객님 기운받아...,긍정,0.910900
16,"맛있어요, 먹었습니다, 먹느라, 따로, 걷어놨는데, 바삭하니, 많고, <menu>를...",안녕하세요 고객님 <menu>를 안 올려드린 줄 알고 놀랐네요 바삭하게 잘 준비 되...,긍정,0.764575
...,...,...,...,...
2988,"맛있었습니다, 싸서먹는, 먹으니, 맛있대요, 야들야들, 부드럽고, 애들은, 괜찮네요...",맛있게 드신것이 글속에 묻어나요 고마워요!~ 다음에도 고객님의 만족을위해 노력하는...,긍정,0.586047
2990,"맛있어요, <menu><menu>, 완전",소중한 리뷰작성 진심으로 감사드립니다 저희 <store>을 찾아주셔서 너<menu>...,긍정,0.667130
2992,"먹었어요, 맛있게, <menu>양이, 한가득입니다, <menu>, 올라가는, 위에, 기본",안안녕하세요 <store> <region> 입니다 D 앞으로도 좋은 재료와 서비스로...,긍정,0.523967
2997,"사장님, <menu>창하세요, 맛있게, 먹었습니다, 주셔도, 남나요, 이렇게",안녕하세요 부자집 입니다! 주문해주셔서 감사합니다 바쁘신 와중에 별점5점!리뷰 감사...,긍정,0.898623


In [108]:
train.replace(['<menu><menu>','<menu> <menu>'], '<menu>')

,review,reply,sentiment,score
0,"입니다, 만족스러운, 제품",감사합니다,긍정,0.572451
1,"<menu>, 날에따라, 만족하는, 대체적으로, 있지만, 약간, 편차가",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,긍정,0.517167
2,"그정도로, 맛있어요, 시켜먹었네요, 이틀연속으로",<store><menu>을 믿고 주문해주셔서 감사합니다 다음 주문시 더욱 더 좋은 ...,긍정,0.791717
3,"좋아요, 괜찮았습니다, <menu>",맛있게 드셨나요?! 고객님들에게 자꾸만 생각나고 떠오르는 맛집으로 기억 속에 남고 ...,긍정,0.380454
4,"메뉴도, 맛있어서, 추천합니다, 여기서만, 여기가, 다른, 먹어요, 존맛탱, <st...",<store> <menu> <store>탕 <menu> 요리 <region> 입니...,긍정,0.814542
...,...,...,...,...
2995,"맛나게, 잘먹엇슴돠, 감사함돠",저희 목살 맛있게 잘 드셨다니 기쁘고 힘이 납니다. 소중한 리뷰와 저희에게 큰 힘이...,긍정,0.580561
2996,"맛있고, 빨랐어요, 배달도",믿고 찾아주시고 맛있게 드셔주셔서 감사합니다! 고객님을 향한 열정만큼은 절대 변함없...,긍정,0.616511
2997,"사장님, <menu>창하세요, 맛있게, 먹었습니다, 주셔도, 남나요, 이렇게",안녕하세요 부자집 입니다! 주문해주셔서 감사합니다 바쁘신 와중에 별점5점!리뷰 감사...,긍정,0.898623
2998,"굳굳, 맛있네요",맛있게 드시고 별 다섯개 만점 리뷰 정말 감사드립니다~,긍정,0.580293


#### 기타표현 처리

- 영어, 오타, 띄어쓰기 청.춘.식.당 / 100g / 이벤트 / 24시간 / 장소 / 영업시간 등

In [ ]:
train

## 증강

In [109]:
from sklearn.utils import resample

minority_class = train[train['sentiment'] == '부정']


In [110]:
mask_neg = train['sentiment'] == '부정'
train[mask_neg].shape[0]

125

In [111]:
mask_pos = train['sentiment'] == '긍정'
train[mask_pos].shape[0]

2875

In [112]:
google_food = pd.read_csv(f"{DATA_PATH}playstore_food_reply_0904.csv")

In [113]:
google_food

,Unnamed: 0,review,score,reply
0,971,<store> 쿠폰 5월내내 하는거 아닌가요? 왜 종료라고 뜨나요?,3,"님, 안녕하세요. <store>입니다. 이용 시 불편을 드려 죄송합니다. 5 9 ..."
1,3104,다이어드,5,"고객님, 안녕하세요. <store>입니다. <store>을 통해 입맛에 맞는 식사로..."
2,3296,야미,5,"Play 고객님, 안녕하세요. <store>입니다. 입맛에 맞는 식사를 하셨다니 ..."
3,4256,맛나는 아이스크림,5,"고객님, 안녕하세요. <store>입니다. 입맛에 맞는 식사를 하셨다니 감사의 말..."
4,5552,다이어트 할거야,1,"고객님, 안녕하세요. <store>입니다. 고객님의 소중한 리뷰 감사드립니다. <s..."
...,...,...,...,...
2682,103578,세상살기좋아졋습니다 이제 밖에서 114에 물어볼필요도없이 바로 가게 실행시키고 주문...,5,안녕하세요. 좋은 세상만드는 <store>입니다. 114는 가르쳐주지 않는 메뉴 이...
2683,103585,이 가게은괜찮음,5,안녕하세요. 완전 좋은 <store>입니다. 고객님들의 애정어린 관심과 격려로 쭉쭉...
2684,103591,깔끔해서 보기도 좋구 찾기쉬움 좋아용~,5,안녕하세요. 보기 좋은 가게 <store>입니다. 카테고리로 분류하여 큼지막한 버튼...
2685,103592,"다른가게대비 제일 정확하고, 등록된 음식점도 많음! 강추!",5,안녕하세요. 강추! <store>입니다. 다른 가게과 비교하기보다 어떻게 하면 고객...


In [114]:
desired_samples = train[mask_pos].shape[0] - train[mask_neg].shape[0] #pos, neg개수 동일하게 하기 위해서
oversampled_data = resample(minority_class, replace=True, n_samples=desired_samples, random_state=42)


In [115]:
aug_train = pd.concat([train,oversampled_data],axis=0)
aug_train.shape

(5750, 4)

## 모델 학습

In [116]:
train_ft = aug_train

In [117]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "skt/kogpt2-base-v2"
# model_name = "EasthShin/Youth_Chatbot_Kogpt2-base"

model = AutoModelForCausalLM.from_pretrained(model_name)

In [118]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

tokenizer.eos_token_id # eos 토큰의 idx 반환

1

In [119]:
tokenizer.tokenize(train['review'][0])

['▁입', '니', '다,', '▁만족', '스러운', ',', '▁제품']

- 데이터셋 만들기

In [120]:
train_ft = train_ft[['review','reply']]

In [121]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.review = df["review"].tolist()
        self.reply = df["reply"].tolist()

    def __len__(self):
        return len(self.review)

    def __getitem__(self,idx):
        return "<w>" + "</s><q>" + self.review[idx] + "</s><a>" + self.reply[idx] + "</s>"

In [122]:
def collate_fn(batch):
    x = tokenizer(batch, return_tensors="pt", padding=True)
    return {"x":x}

In [123]:
train_dt = ChatDataset(train_ft)

In [124]:
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=2,collate_fn=collate_fn)

In [125]:
batch = next(iter(train_dl))
batch

{'x': {'input_ids': tensor([[ 9724,   461,   405,     1,  9724,   455,   405,  8153,  7172, 14364,
          13902, 12512,   387, 10395,     1,  9724,   439,   405, 21868,  8708,
           7172,  7182,   739,     1,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3],
         [ 9724,   461,   405,     1,  9724,   455,   405,   403,   451,  97

In [126]:
tokenizer.pad_token_id # 패딩토큰인 3번 빼고 계산해야겠군

3

- 학습 loop함수화

In [127]:
def train_loop(dataloader,model,optimizer,loss_fn,device):
    epoch_loss = 0
    model.train()

    for batch in tqdm(dataloader):
        x = batch["x"].to(device)
        pred = model(**x).logits # batch, seq, 단어별 실수값
        n_class = pred.shape[-1] # 클래스 개수
        pred = pred[:,:-1] # eos 토큰 제외
        pred = pred.reshape(-1,n_class) # 2차원 형태로 변환

        tgt = x["input_ids"][:,1:]
        tgt = tgt.flatten() # 1차원 벡터 형태로 변환

        mask = tgt != 3 # 패딩토큰인 3번 빼고 계산해야겠군
        tgt = tgt[mask]
        pred = pred[mask]
        loss = loss_fn(pred,tgt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    return epoch_loss

- 학습하기

In [128]:
batch_size = 2
epochs = 10
loss_fn = torch.nn.CrossEntropyLoss()

In [129]:
train_ft['review'] = train_ft['review'].astype(str)
train_ft['reply'] = train_ft['reply'].astype(str)
# train_ft['store'] = train_ft['store'].astype(str)

<ipython-input-129-52e78ab09a80>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ft['review'] = train_ft['review'].astype(str)
<ipython-input-129-52e78ab09a80>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ft['reply'] = train_ft['reply'].astype(str)


- 총 5750개의 데이터 학습 (증강)

In [130]:
is_holdout = False
reset_seeds(SEED)

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=3e-5)

train_dt = ChatDataset(train_ft)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for i in range(epochs):
    train_loss = train_loop(train_dl, model, optimizer, loss_fn, device)
    print(train_loss)

    if is_holdout:
        break


# 0.18275894721305888

100%|██████████| 2875/2875 [06:19<00:00,  7.57it/s]


1.2372494839818582


100%|██████████| 2875/2875 [06:19<00:00,  7.57it/s]


0.6706806768464005


100%|██████████| 2875/2875 [06:23<00:00,  7.50it/s]


...

0.23829685253617555


100%|██████████| 2875/2875 [06:12<00:00,  7.72it/s]


0.2046047156219897


100%|██████████| 2875/2875 [06:13<00:00,  7.69it/s]

0.18275894721305888


- 모델 저장하기

In [133]:
model.save_pretrained(f"{DATA_PATH}koGPT2_keybert_masktoken_0905") #0

## 답변 생성

- 맞춤법 처리

In [141]:
!git clone https://github.com/ssut/py-hanspell.git

Cloning into 'py-hanspell'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 110 (delta 16), reused 9 (delta 9), pack-reused 88
Receiving objects: 100% (110/110), 22.30 KiB | 4.46 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [142]:
!cd py-hanspell

In [143]:
!pip install py-hanspell/

Processing ./py-hanspell
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4813 sha256=9fd18c96fc9d1ca9100ff1c92ce02a4743bdff90d958184dd4c209a5ec1b4f52
  Stored in directory: /root/.cache/pip/wheels/a3/a7/16/8db596361f402638188412108152b8fbf6db76c898e00fe02d
Successfully built py-hanspell


In [144]:
from hanspell import spell_checker

# def correct_spell(text):
#     if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
#         corrected_text = spell_checker.check(text).checked
#         return corrected_text
#     else:
#         return text

- 모델 불러오기

In [145]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# model_name = 'MLOpsEngineer/review-rate-prediction'

# model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}ygy_cont3000_youth_ver1")
model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}koGPT2_keybert_masktoken_0905")

In [146]:
model_name = "skt/kogpt2-base-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name, #해당 모델의 토크나이저
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024,
                                          padding_side='left')

In [147]:
def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

In [ ]:
# @torch.no_grad()
# def chatbot(model,tokenizer,max_len,device):
#     model.eval()
#     while True:
#         text = input("소비자 > ").strip()
#         if text == "quit":
#             break
#         text = "<q>" + text + "</s><a>"
#         x = tokenizer.encode(text,return_tensors="pt").to(device) # batch, seq

#         result_ids = model.generate(x,
#                             max_length=max_len,
#                             repetition_penalty=2.0,
#                             use_cache=True,
#                             do_sample=True, # 확률적 샘플링 여부(False = greedy 방식)
#                             temperature=0.2, # 소프트맥스 온도(확률적 샘플링일 때만 작동)
#                             top_k=5) # 상위 확률 k개 기준으로 확률적 샘플링 하겠다(확률적 샘플링일 때만 작동)
#         q_len = len(text) + 1
#         text = tokenizer.decode(result_ids[0])

#         from hanspell import spell_checker

#         text = correct_spell(text)

#         # if len(text) > max_len:
#         #     text = text[:max_len]

#         print("사장님 > ",text[q_len:-4])

In [173]:

# chatbot 함수 정의
def chatbot(model, tokenizer, max_len, review_list, store, menu, region):
    responses = []  # 결과를 저장할 리스트 초기화

    for review in tqdm(review_list):

        review = review.replace('<store>',store)
        review = review.replace('<menu>',menu)
        review = review.replace('<region>',region)


        # 대화 시작 및 종료 토큰 추가
        text = "<q>" + review + "</s><a>"

        # 입력 텍스트를 인코딩
        x = tokenizer.encode(text, return_tensors="pt")
        # max_len = 10000, max_length=max_len

        # 모델로부터 응답 생성
        result_ids = model.generate(x,
                                    max_length=max_len,
                                    num_return_sequences=1,
                                    repetition_penalty=0.9, # 1보다 낮으면 반복 허용, 1보다 클수록 창의적
                                    use_cache=True,
                                    do_sample=True, # 확률적 샘플링 여부(False = greedy 방식)
                                    temperature=1, # softmax. 1보다 낮으면 결정적, 1보다 클수록 창의적
                                    top_k=5) # 상위 확률 k개 기준으로 확률적 샘플링 하겠다(확률적 샘플링일 때만 작동)


        # 응답을 텍스트로 디코딩하여 저장
        response = tokenizer.decode(result_ids[0], skip_special_tokens=True)

        response = response.replace('<store>',store)
        response = response.replace('<menu>',menu)
        response = response.replace('<region>',region)

        response = response.split("<a>", 1)[1]
        response = correct_spell(response)
        responses.append(response)

    # 결과를 데이터프레임으로 변환
    result_df = pd.DataFrame({"Review": review_list, "Response": responses})
    return result_df


In [179]:
review_list = [

    '맛이 있어요',
    '최고에요',
    '사장님이 친절하시네요',
    '맛은 있는데 배달이 식어서 왔어요ㅠㅠ',
    '맛이 없어요',
    '이 머리카락 들어있는거 뭔가요?',
    '배달이 너무 불친절해요',
    '최악이에요'

]

# chatbot 실행 및 결과 출력
result_df = chatbot(model, tokenizer, 512, review_list,'최가네','덮밥','서초점')

pd.DataFrame(result_df)

100%|██████████| 8/8 [00:25<00:00,  3.19s/it]


,Review,Response
0,맛이 있어요,안녕하세요 고객님 맛있게 드셔주시고 리뷰까지 너무 고맙습니다 항상 신선한 재료로 정...
1,최고에요,"저희 최가네 청주점을 찾아주셔서 너무 감사드려요! 더 발전하는 모모가 되겠습니다, ..."
2,사장님이 친절하시네요,주문 주셔서 감사드립니다 요즘 덮밥이 핫한 메뉴이네요 감사합니다! 더 맛있는 음식 ...
3,맛은 있는데 배달이 식어서 왔어요ㅠㅠ,안녕하세요 최가네입니다. 맛있게 드셨다니 저희도 기분이 좋습니다 정성스러운 리뷰도 ...
4,맛이 없어요,맛있게 드셔주셔서 너무 감사드립니다
5,이 머리카락 들어있는거 뭔가요?,주문해 주셔서 감사합니다. 언제나 맛있게 정성스럽게 만들어 보내드립니다. 맛있는 덮...
6,배달이 너무 불친절해요,맛있게 드셔 주시고 소중한 리뷰 남겨주셔서 감사합니다~ 만족스러운 식사하신 것 같아...
7,최악이에요,너무 감사합니다 항상 최선을 다해 모시겠습니다~


In [176]:
review_list = [

    '맛이 있어요',
    '최고에요',
    '사장님이 친절하시네요',
    '맛은 있는데 배달이 식어서 왔어요ㅠㅠ',
    '맛이 없어요',
    '이 머리카락 들어있는거 뭔가요?',
    '배달이 너무 불친절해요',
    '최악이에요'

]

# chatbot 실행 및 결과 출력
result_df = chatbot(model, tokenizer, 512, review_list,'비비큐','치킨','양재점')

pd.DataFrame(result_df)

100%|██████████| 8/8 [00:44<00:00,  5.53s/it]


,Review,Response
0,맛이 있어요,맛있게 드셔주셔서 감사합니다~ 요청사항은 조리 전 늘 꼼꼼하게 체크하고 있으니 필요...
1,최고에요,저희 비비큐 양재점을 찾아주셔서 정말 대단히 감사합니다 이렇게 맛있게 드셔주셨다니 ...
2,사장님이 친절하시네요,"고객님, 안녕하세요 시간 내어 리뷰 작성해 주셔서 정말 감사합니다 항상 고객님들 주..."
3,맛은 있는데 배달이 식어서 왔어요ㅠㅠ,안녕하세요 비비큐입니다. 맛있게 드셨다니 저희도 기분이 좋습니다 정성스러운 리뷰도 ...
4,맛이 없어요,안녕하세요~! 푸짐하고 맛있는 비비큐입니다 맛있게 잘 드셔주신 것 같아 기분 좋습니...
5,이 머리카락 들어있는거 뭔가요?,맛있는 음식이 필요한 순간에 저희 비비큐 복대점 을 선택해 주셔서 대단히 감사드립니...
6,배달이 너무 불친절해요,맛있게 드셔 주시고 소중한 리뷰 남겨주셔서 감사합니다~ 만족스러운 식사하신 것 같아...
7,최악이에요,너무 감사합니다 늘 변함없는 맛과 양으로 보답 드리겠습니다~! 치킨 음식 생각나실 ...


In [177]:
train_ft.shape # 2800-900개

(5750, 2)

In [ ]:
result_df.to_csv(f'{DATA_PATH}[response]koGPT2_keybert_masktoken_0905_1.csv',index=False)

In [165]:
review_list = [
    # 특정 메뉴명 포함 (맛)
    '이집 피자 최애 넘나 맛있어요',
    '치킨이 바삭바삭하니 맛있네요',
    '마라샹궈 넘넘 맛있고 양도 푸짐해용! 감사합니다!',
    '간만에 술한잔생각나서 연어 주문해서 먹는데 맛있네요^^ 양이 좀적어진거같은데 다음에는 추가로 시켜야 겠어요^^ 잘먹었습니당^^',
    '짜장면 배달이 너무 느리게 와서 음식이 다 불었네요..사장님 신경좀 써주세요',
    '쌀국수 먹는데 비닐 나왔네요 반품 이런건 안하지만 청결에 신경 써주세요',
    '요기 김치찜 맛있는데 가끔 좀 짜요ㅠㅠ다음엔 육수 추가로 주심 좋겠어요',

    # 메뉴의 특성 포함 (맛)
    '닭도 쫄깃하니 맛있고 떡볶이도 맛나요👍',
    '오늘은 비계가 많은거 같아요.. 양이 상대적으로 적어서 아쉬웠어요.',
    '크림이 너무 부드럽고 맛있어요, 또 시켜먹을게요!!',
    '맛있게 먹었어요 4단계로 해도 맵지는 않아요',


    # 배달 및 기타 코멘트
    '배달 빠르게 와서 맛있게 먹었습니다!',
    '완전 만족했어요 전문 블로거인데요 양쪼끔만 더 많았으면! 종종 시켜먹을게요',
    '맛은 정말 좋았지만, 편의점가려다 배달 도착한걸 발견 못해다면 다 상한 음식 먹을뻔했네요. 노크도 없고 문자도 안주시면 음식이 온지 어떻게 알으라는건지....',
    '.................',
    '벌레 나왔어요.. 다신 안시켜먹을 것 같아요',
]

result_df = chatbot(model, tokenizer, 1024, review_list,'비비큐','치킨','서초점')

pd.DataFrame(result_df)

100%|██████████| 16/16 [01:03<00:00,  3.99s/it]


,Review,Response
0,이집 피자 최애 넘나 맛있어요,"고객님, 안녕하세요! 항상 최선을 다해 모시겠습니다! 주문해 주셔서 감사합니다!"
1,치킨이 바삭바삭하니 맛있네요,안녕하세요 저희 매장은 비비큐 가족이 먹는다는 마음가짐으로 항상 맛과 양뿐만이 아니...
2,마라샹궈 넘넘 맛있고 양도 푸짐해용! 감사합니다!,항상 웃음이 가득한 비비큐을 찾아주셔서 감사합니다 언제나 내가 먹는 음식이라고 생각...
3,간만에 술한잔생각나서 연어 주문해서 먹는데 맛있네요^^ 양이 좀적어진거같은데 다음에...,"사진과 함께 좋은 평점 주셔서 감사드리며, 항상 맛있는 음식과 빠른 배달로 식사시간..."
4,짜장면 배달이 너무 느리게 와서 음식이 다 불었네요..사장님 신경좀 써주세요,소중한 리뷰 감사합니다 저희 음식으로 만족스러운 식사시간이 되신 것 같아 뿌듯하네요...
5,쌀국수 먹는데 비닐 나왔네요 반품 이런건 안하지만 청결에 신경 써주세요,주문해 주셔서 감사합니다. 늘 한결같은 마음과 맛으로 정성을 다하겠습니다. 맛있는 ...
6,요기 김치찜 맛있는데 가끔 좀 짜요ㅠㅠ다음엔 육수 추가로 주심 좋겠어요,고객님 소중한 리뷰 감사드립니다. 소중한 의견 감사드립니다. 맛있게 식사하셨다니 다...
7,닭도 쫄깃하니 맛있고 떡볶이도 맛나요👍,항상 웃음이 가득한 비비큐을 찾아주셔서 감사합니다 언제나 내가 먹는 음식이라고 생각...
8,오늘은 비계가 많은거 같아요.. 양이 상대적으로 적어서 아쉬웠어요.,안녕하세요 고객님 비계량도 너무 적어서 정말 죄송합니다. 매장에서 드시는 식사시간은...
9,"크림이 너무 부드럽고 맛있어요, 또 시켜먹을게요!!",맛있게 드셔주셔서 너무 감사드립니다! 언제나 맛있게 드셔주셔서 감사드립니다! 언제나...


In [ ]:

chatbot(model,tokenizer,128,device) # quit

# --------참고. 리뷰 키워드 사전학습모델

## A. Keybert로 키워드 추출

In [ ]:
!pip install keybert

### 1) skt/kobert-base-v1

In [ ]:
# from keybert import KeyBERT
# # from transformers import XLNetTokenizer, BertTokenizer, BertModel
# from tqdm import tqdm  # tqdm 추가
# from transformers import AutoModel, AutoTokenizer


# # keybert에 사용할 Kobert 모델 로드
# tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
# model = AutoModel.from_pretrained("skt/kobert-base-v1")

# # KeyBERT 모델 초기화
# keybert_model = KeyBERT(model=model)

# # 리뷰 데이터 (train['review']로 가정)
# reviews = train['review'][:50]  # train['review']에 리뷰 데이터를 넣으세요
# keywords_list = []

# # tqdm을 사용하여 진행 상황 표시
# for text in tqdm(reviews):
#     keywords = keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
#     keywords_list.append(keywords)

# # # 결과 출력
# # for idx, keywords in enumerate(keywords_list):
# #     print(f"Review {idx + 1} Keywords:", keywords)

# result_df = pd.DataFrame({'Review': reviews, 'Keywords': keywords_list})
# result_df




### 2) BM-K/KoSimCSE-roberta

In [ ]:
# import pandas as pd
# from keybert import KeyBERT
# from tqdm import tqdm
# # from transformers import BertTokenizer, RobertaTokenizer, RobertaModel
# from transformers import AutoModel, AutoTokenizer



# # BM-K/KoSimCSE-roberta 모델 로드
# tokenizer = AutoTokenizer.from_pretrained("BM-K/KoSimCSE-roberta")
# model = AutoModel.from_pretrained("BM-K/KoSimCSE-roberta")

# # KeyBERT 모델 초기화
# keybert_model = KeyBERT(model=model)

# # 리뷰 데이터 (train['review']로 가정)
# reviews = train['review'][:50]  # train['review']에 리뷰 데이터를 넣으세요
# keywords_list = []

# # tqdm을 사용하여 진행 상황 표시
# for text in tqdm(reviews):
#     keywords = keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=5)
#     keywords_list.append(keywords)

# # 데이터 프레임으로 변환
# result_df = pd.DataFrame({'Review': reviews, 'Keywords': keywords_list})
# result_df


### 3) distilbert-base-nli-mean-tokens
- 이 모델이 Keybert의 디폴트 모델인거 같음

In [ ]:
# from keybert import KeyBERT
# from tqdm import tqdm


# # KeyBERT 모델 초기화 (distilbert-base-nli-mean-tokens 사용)
# keybert_model = KeyBERT('distilbert-base-nli-mean-tokens')

# # 리뷰 데이터 (train['review']로 가정)
# reviews = train['review'][:50]  # train['review']에 리뷰 데이터를 넣으세요
# keywords_list = []

# # tqdm을 사용하여 진행 상황 표시
# for text in tqdm(reviews):
#     keywords = keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=5)
#     keywords_list.append(keywords)

# # 결과 출력
# result_df = pd.DataFrame({'Review': reviews, 'Keywords': keywords_list})
# result_df


### 4) monologg/kobert

In [ ]:
# import pandas as pd
# from keybert import KeyBERT
# from tqdm import tqdm
# # from transformers import BertTokenizer, RobertaTokenizer, RobertaModel
# from transformers import AutoModel, AutoTokenizer



# # BM-K/KoSimCSE-roberta 모델 로드
# tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
# model = AutoModel.from_pretrained("monologg/kobert")

# # KeyBERT 모델 초기화
# keybert_model = KeyBERT(model=model)

# # 리뷰 데이터 (train['review']로 가정)
# reviews = train['review'][:50]  # train['review']에 리뷰 데이터를 넣으세요
# keywords_list = []

# # tqdm을 사용하여 진행 상황 표시
# for text in tqdm(reviews):
#     keywords = keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
#     keywords_list.append(keywords)

# # 데이터 프레임으로 변환
# result_df = pd.DataFrame({'Review': reviews, 'Keywords': keywords_list})
# result_df



## B. 키위로 키워드 추출